In [ ]:
! pip install dataset transformers

In [ ]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
checkpoint = "bert-base-uncased"

tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=TFAutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)


In [ ]:
tokenizer('i like meet','i hate food')

In [ ]:
dataset = load_dataset("csv", data_files="/kaggle/input/question-pairs-dataset/questions.csv",encoding='ISO-8859-1',split='train')                    

In [ ]:
dataset

In [ ]:
dataset.features

In [ ]:
dataset[:10]

In [ ]:
dataset=dataset.rename_column('is_duplicate','label')
dataset

In [ ]:
dataset=dataset.remove_columns(['id', 'qid1', 'qid2'])

In [ ]:
dataset[:10]

In [ ]:
dataset

In [ ]:
count_label=np.bincount(dataset['label'])
count_label

In [ ]:
import matplotlib.pyplot as plt

categories = ['duplicates', 'non_duplicate']

x = range(len(categories))

plt.bar(x, count_label)

plt.xlabel('Categories')
plt.ylabel('Values')
plt.title('Bar Chart')

plt.xticks(x, categories)

plt.show()

In [ ]:
min_label=min(count_label)
min_label

In [ ]:
count_non_dublicate=np.where(np.array(dataset['label'])==0)[0][0:min_label]
count_dublicate=np.where(np.array(dataset['label'])==1)[0][0:min_label]
balance_index=np.concatenate([count_non_dublicate,count_dublicate],axis=0)

In [ ]:
dataset=dataset.select(balance_index)
np.bincount(dataset['label'])

In [ ]:
del count_non_dublicate  ,count_dublicate,balance_index

In [ ]:
dataset

In [ ]:
dataset['question2']

In [ ]:
dataset=dataset.filter(lambda x:x['question1'] is not None)
dataset=dataset.filter(lambda x:x['question2'] is not None)

In [ ]:
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str('How would you find the zero of the function for [math]y=\\frac{4x^2 - 36x}{ x-9}[/math]?' ))

In [ ]:
%%time
dataset=dataset.map(lambda x:tokenizer(x['question1'],x['question2'],truncation=True),batched=True)
dataset[0:10]


In [ ]:
data_colletor=DataCollatorWithPadding(tokenizer=tokenizer,return_tensors='tf')


In [ ]:
dataset=dataset.train_test_split(shuffle=True,test_size=.03)
dataset['validation']=dataset['train'].train_test_split(shuffle=True,test_size=.07)['test']
dataset

In [ ]:
train_data=dataset['train']
validation_data=dataset['validation']
test_data=dataset['test']

In [ ]:
train_data=train_data.to_tf_dataset(columns=['input_ids', 'token_type_ids', 'attention_mask'],
    batch_size=16,
    shuffle=True,
    collate_fn=data_colletor,
    label_cols=['label'])
validation_data=validation_data.to_tf_dataset(columns=['input_ids', 'token_type_ids', 'attention_mask'],
    batch_size=16,
    shuffle=True,
    collate_fn=data_colletor,
    label_cols=['label'])
test_data=test_data.to_tf_dataset(columns=['input_ids', 'token_type_ids', 'attention_mask'],
    batch_size=16,
    shuffle=False,
    collate_fn=data_colletor,
    label_cols=['label'])

In [ ]:
from transformers import create_optimizer

num_epochs = 2
num_train_steps = len(train_data) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)
model.fit(
    train_data,epochs=num_epochs,validation_data=validation_data
)

In [ ]:
prediction=model.predict(test_data)['logits']
prediction=np.argmax(prediction,axis=-1)
accuracy=np.sum(prediction==dataset['test']['label'])/len(prediction)
print(f'The Acuuracy Is: {accuracy}')

In [ ]:
class_names=['Non_Duplication','Duplication']
confusion = confusion_matrix(prediction,dataset['test']['label'])
sns.heatmap(confusion, annot=True, cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()

In [ ]:
def prediction(sample):
    encoded_sample = tokenizer.encode(sample,truncation=True)
    input_data = np.array([encoded_sample])
    prediction=model.predict(input_data)['logits']
    pred=np.argmax(prediction,axis=-1)
    print(pred)
    label='Non_Duplication'if pred[0]==0 else 'Duplication'
    return f'This Email Is: {label}'

In [ ]:
print(dataset['test']['question1'][1],dataset['test']['question2'][1])
print(dataset['test']['label'][1])
sample=dataset['test']['question1'][1],dataset['test']['question2'][1]
print(prediction(sample))

In [ ]:
print(dataset['test']['question1'][10],dataset['test']['question2'][10])
sample=dataset['test']['question1'][10],dataset['test']['question2'][10]
print(dataset['test']['label'][10])
print(prediction(sample))